We will use the Motor Trend Cars Dataset to explore PySpark.

The data was extracted from the 1974 Motor Trend US magazine, and comprises fuel consumption 
and 10 aspects of automobile design and performance for 32 automobiles (1973–74 models).

The dataset can be downloaded from https://gist.githubusercontent.com/seankross/a412dfbd88b3db70b74b/raw/5f23f993cd87c283ce766e7ac6b329ee7cc2e1d1/mtcars.csv


In [1]:
from pyspark.sql import *
import pyspark
spark = pyspark.SQLContext(sc)
import pandas as pd

In [16]:
!curl -O https://gist.githubusercontent.com/seankross/a412dfbd88b3db70b74b/raw/5f23f993cd87c283ce766e7ac6b329ee7cc2e1d1/mtcars.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1700  100  1700    0     0  10658      0 --:--:-- --:--:-- --:--:-- 14529


In [17]:
!head mtcars.csv

model,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
Mazda RX4,21,6,160,110,3.9,2.62,16.46,0,1,4,4
Mazda RX4 Wag,21,6,160,110,3.9,2.875,17.02,0,1,4,4
Datsun 710,22.8,4,108,93,3.85,2.32,18.61,1,1,4,1
Hornet 4 Drive,21.4,6,258,110,3.08,3.215,19.44,1,0,3,1
Hornet Sportabout,18.7,8,360,175,3.15,3.44,17.02,0,0,3,2
Valiant,18.1,6,225,105,2.76,3.46,20.22,1,0,3,1
Duster 360,14.3,8,360,245,3.21,3.57,15.84,0,0,3,4
Merc 240D,24.4,4,146.7,62,3.69,3.19,20,1,0,4,2
Merc 230,22.8,4,140.8,95,3.92,3.15,22.9,1,0,4,2


We read the data into a Spark dataframe.

In [5]:
mtcars = sqlContext.read.csv('mtcars.csv', header=True, inferSchema=True)

If we wish, we can convert the Spark dataframe to a pandas dataframe. But a pandas dataframe must fit in memory, while Spark is generally used to work with datasets that are too large to fit in memory. Our toy example has only 32 rows. 
¯\\_(ツ)_/¯

In [6]:
df = mtcars.toPandas()
df.head()

,model,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
0,Mazda RX4,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4
1,Mazda RX4 Wag,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4
2,Datsun 710,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1
3,Hornet 4 Drive,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1
4,Hornet Sportabout,18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2


In [45]:
mtcars.groupBy('cyl', 'am').avg("mpg").orderBy('cyl', 'am').show()

+---+---+------------------+
|cyl| am|          avg(mpg)|
+---+---+------------------+
|  4|  0|22.900000000000002|
|  4|  1|28.075000000000003|
|  6|  0|            19.125|
|  6|  1|20.566666666666666|
|  8|  0|15.050000000000002|
|  8|  1|              15.4|
+---+---+------------------+



In [47]:
mtcars.select('model', 'mpg').take(5)

[Row(model='Mazda RX4', mpg=21.0),
 Row(model='Mazda RX4 Wag', mpg=21.0),
 Row(model='Datsun 710', mpg=22.8),
 Row(model='Hornet 4 Drive', mpg=21.4),
 Row(model='Hornet Sportabout', mpg=18.7)]

In [10]:
trans = spark.createDataFrame([{'am': 0, 'transmission': 'automatic'}, {'am': 1, 'transmission': 'manuak'}])

/usr/local/bin/spark/python/pyspark/sql/session.py:320: UserWarning: inferring schema from dict is deprecated,please use pyspark.sql.Row instead
  warnings.warn("inferring schema from dict is deprecated,"


In [14]:
df = mtcars.join(trans, mtcars.am == trans.am).drop('am')
df.show(10)

+-------------------+----+---+-----+---+----+-----+-----+---+----+----+------------+
|              model| mpg|cyl| disp| hp|drat|   wt| qsec| vs|gear|carb|transmission|
+-------------------+----+---+-----+---+----+-----+-----+---+----+----+------------+
|   Pontiac Firebird|19.2|  8|400.0|175|3.08|3.845|17.05|  0|   3|   2|   automatic|
|         Camaro Z28|13.3|  8|350.0|245|3.73| 3.84|15.41|  0|   3|   4|   automatic|
|        AMC Javelin|15.2|  8|304.0|150|3.15|3.435| 17.3|  0|   3|   2|   automatic|
|   Dodge Challenger|15.5|  8|318.0|150|2.76| 3.52|16.87|  0|   3|   2|   automatic|
|      Toyota Corona|21.5|  4|120.1| 97| 3.7|2.465|20.01|  1|   3|   1|   automatic|
|  Chrysler Imperial|14.7|  8|440.0|230|3.23|5.345|17.42|  0|   3|   4|   automatic|
|Lincoln Continental|10.4|  8|460.0|215| 3.0|5.424|17.82|  0|   3|   4|   automatic|
| Cadillac Fleetwood|10.4|  8|472.0|205|2.93| 5.25|17.98|  0|   3|   4|   automatic|
|        Merc 450SLC|15.2|  8|275.8|180|3.07| 3.78| 18.0|  0|   3